In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

import sys
sys.path.append('Config')
from lablib import *

In [ ]:
root_path = r'Dati/Parte 2/Caratterizzazione_impulso/'
skip = 18

def read_data(filename, titles = ['Time', 'V']):
    df = read_corretto(root_path + filename + '/dat.csv', skip, titles)

    # Sostituire tutte le istanze di "," con "" nelle colonne: 'V', 'Time'
    df['V'] = df['V'].str.replace(",", "", case=False, regex=False)
    df['Time'] = df['Time'].str.replace(",", "", case=False, regex=False)

    # Convert all columns to float
    df = df.astype(float)

    return df

def quick_plot(df):
    plt.plot(df['Time'], df['V'])
    plt.xlabel('Time')
    plt.ylabel('V')
    plt.title('Signal')
    plt.show()

## BGO

In [ ]:
bgo_na = read_data('BGO_Na')
quick_plot(bgo_na)

In [ ]:
bgo_co = read_data('BGO_Co')
quick_plot(bgo_co)

## CSI

In [ ]:
csi_na = read_data('CsI_Na')
quick_plot(csi_na)

In [ ]:
csi_co = read_data('CsI_Co')
quick_plot(csi_co)

## Lyso

In [ ]:
lyso = read_data('LySO')
quick_plot(lyso)